In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import matplotlib.pyplot as plt

: 

In [3]:
sample_image ="archive/Positive/Labelled/1.jpg"


In [ ]:
image = plt.imread(sample_image)
print(image.shape)

In [ ]:
plt.imshow(image)

In [ ]:
neg_image = plt.imread("archive/Negative/Clear/1.jpg")
plt.imshow(neg_image)

In [ ]:
import os

negative_imgs = len(os.listdir("archive/Negative/Clear"))
positive_imgs = len(os.listdir("archive/Positive/Labelled"))

print("total images = ", (negative_imgs + positive_imgs))

In [ ]:


fig, ax = plt.subplots()

fruits = ['positive images', 'negative images']
counts = [positive_imgs, negative_imgs]
bar_labels = ['positive images', 'negative images']
bar_colors = ['tab:red', 'tab:blue']

ax.bar(fruits, counts, label=bar_labels, color=bar_colors)

ax.set_ylabel('Count')
ax.set_title('Sickle Cell Data Distribution')
ax.legend()


In [8]:
data_df = pd.DataFrame(columns = ["Image dir", "Label"])

In [9]:
positive_img_dir = "archive/Positive/Labelled"
negative_img_dir ="archive/Negative/Clear"

In [10]:
import os
import re
import cv2

Images = []
Labels = []

#append in the  postive Images
for file_dir, _, files in os.walk(positive_img_dir):
    for file in files:
        img_file = os.path.join(file_dir, file)
        img = cv2.imread(img_file)
        img = cv2.resize(img, (255, 255))
        Images.append(img)
        Labels.append(1)

In [11]:
#append in the negative Images
for file_dir, _, files in os.walk(negative_img_dir):
    for file in files:
        img_file = os.path.join(file_dir, file)
        img = cv2.imread(img_file)
        img = cv2.resize(img, (255, 255))
        Images.append(img)
        Labels.append(0)

In [ ]:
Images = np.array(Images)

Images.shape

In [ ]:
sample = Images[0]

plt.imshow(sample)
plt.title("Postive")

In [ ]:
sample = Images[-1]
plt.imshow(sample)
plt.title("Negative")

# train the model

In [15]:
X = Images / 255 #scale the data
y = Labels

In [16]:
from sklearn.model_selection import train_test_split

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, np.array(y), stratify = np.array(y),
                                                    random_state = 42,
                                                    shuffle = True, test_size = 0.2)

train_X.shape, test_X.shape, train_y.shape, test_y.shape

In [ ]:
np.unique(train_y, return_counts = True) , np.unique(test_y, return_counts = True)

# Model training

In [ ]:
from tensorflow import keras

In [20]:
# Importing all necessary libraries
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

img_width, img_height = 255, 255
input_shape = (img_width, img_height, 3)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [ ]:
model.summary()

In [23]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
# Add a callback to avoid overfitting

model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath = "best_model.keras",
    monitor='val_accuracy',
    save_best_only=True
)

In [ ]:
model.fit(train_X, train_y, validation_data = (test_X, test_y),
          epochs = 10, callbacks = [model_checkpoint])

In [26]:
best_model = keras.models.load_model("best_model.keras")

# Test the model

In [ ]:
predictions = best_model.predict(test_X)

In [28]:
predictions = np.round(predictions)

In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import seaborn as sns

In [30]:
def get_score(y_pred, y_true):
    print("Accuracy score - ", accuracy_score(y_pred, y_true))
    print("Precision score - ", precision_score(y_pred, y_true))
    print("Recall score - ", recall_score(y_pred, y_true))
    print("f1 score - ", f1_score(y_pred, y_true))
    print("----------  Classification Report   -------")
    print(classification_report(y_pred, y_true))
    
    matrix = confusion_matrix(y_pred, y_true)
    sns.heatmap(matrix, cmap='Blues', annot=True, fmt='d')

In [ ]:
y_pred = model.predict(train_X)
y_pred

In [ ]:
y_pred = np.round(y_pred)
get_score(y_pred, train_y)

In [ ]:
get_score(predictions, test_y)